## 레시피 데이터 정제
* 결측치(CKG_NM) 제거
* 정제 : 난이도(CKG_DODF_NM)
* 난이도 (CKG_DODF_NM) 수치화
  * 아무나 : 1, 초급 : 2, 중급 : 3, 고급 : 4, 최상위 : 5
* 요리명(CKG_NM) 중복 제거
  * 같은 요리명에 대해 난이도, 재료수, 양념수는 평균 사용
  * 하고 나머지 항목은 가장 최근 레시피를 기준으로 한다

### 환경 : import 구문

In [ ]:
import pandas as pd
import logging
import time
from datetime import datetime

### 환경 : 전역 변수 선언

In [ ]:
recipe_preproc_csvfile = 'recipe_preproc.csv'

### 환경 : 로깅 설정

In [ ]:
# 로깅 설정
#logging.basicConfig(
#    level=logging.INFO,
#    format='%(asctime)s - %(levelname)s: %(message)s',
#    handlers=[
#        logging.FileHandler('mealkit_recipe_cleansing_analysis.log'),
#        logging.StreamHandler()
#    ]
#)

## 데이터 정제

### csv파일 dataframe 로딩과 결측치 제거

In [ ]:
try:
  df = pd.read_csv(recipe_preproc_csvfile)
except FileNotFoundError:
  print(f"Error: {recipe_preproc_csvfile} not found.")
  exit()

In [ ]:
##### 결측치 제거
empty_ckg_nm_count = df['CKG_NM'].isnull().sum()
print(f"Number of rows with empty CKG_NM: {empty_ckg_nm_count}")

# 'CKG_NM' 빈 값은 row 자체 제거
df = df.dropna(subset=['CKG_NM'])

Number of rows with empty CKG_NM: 2168


### 데이터 확인

In [ ]:
# prompt: df['CKG_DODF_NM'] == '珂' 것 목록

# Find rows where 'CKG_DODF_NM' is '珂'
filtered_df = df[df['CKG_DODF_NM'] == '珂']

# Print the filtered dataframe (or specific columns if needed)
filtered_df
# Or, to get just a list of the indices:
# print(filtered_df.index.tolist())

,RCP_SNO,CKG_NM,INQ_CNT,RCMM_CNT,SRAP_CNT,CKG_MTH_ACTO_NM,CKG_STA_ACTO_NM,CKG_MTRL_ACTO_NM,CKG_KND_ACTO_NM,CKG_MTRL_CN,CKG_INBUN_NM,CKG_DODF_NM,CKG_TIME_NM,MATERIAL_CNT,SEASONING_CNT
224399,6916639,카레,674,0,8,끓이기,일상,소고기,양념/소스/잼,[재료] 셀러리악 1통| 양파 3개| 당근 2개| 감자 3개| 소고기 어깨살 400...,4인분,珂,NaN,9,0
226952,6919609,고구마크림스튜,1140,0,30,끓이기,일상,채소류,스프,[재료] 무항생제 닭가슴살 100g| 친환경 고구마 2개| 감자 1/2개| 양파 1...,2인분,珂,NaN,8,0


In [ ]:
# 각 값 확인
print(df['CKG_DODF_NM'].unique())

print(df['CKG_INBUN_NM'].unique())

print(df['CKG_TIME_NM'].unique())

['초급' '중급' '아무나' '고급' nan '최상위']
['2인분' '1인분' '3인분' '6인분이상' '4인분' nan '5인분' '2인' '6인분鵑' '2觀' '1觀' '1인' '4觀'
 '6인隙鵑' '3인' '4인' '5인' '6인분이' '6觀隙鵑' '3觀' '5觀']
['60분이내' '30분이내' '15분이내' '90분이내' '2시간이상' '10분이내' nan '5분이내' '2시간이내' '60隙犬'
 '30隙犬' '30분이' '10분이' '15隙犬' '2시＠犬' '90분犬' '15분이' '10분犬' '15분犬' '30분犬'
 '5隙犬' '60분이' '2시간이' '60분犬' '10隙犬' '2시간鵑' '90隙犬' '5분이' '90분이' '5분犬' '2시＠鵑']


### 데이터 수정, 수치화

In [ ]:
df['CKG_DODF_NM'] = df['CKG_DODF_NM'].replace({'초': '초급', '중': '중급', '아무': '아무나', '신의경지': '최상위'})

def update_difficulty(df):
    conditions = [
        (df['CKG_DODF_NM'].isin(['아코', '珂', '틜コ', '薩', '慈', '珂'])) & (df['CKG_TIME_NM'] == '10분이내'),
        (df['CKG_DODF_NM'].isin(['아코', '珂', '틜コ', '薩', '慈', '珂'])) & (df['CKG_TIME_NM'] == '15분이내'),
        (df['CKG_DODF_NM'].isin(['아코', '珂', '틜コ', '薩', '慈', '珂'])) & (df['CKG_TIME_NM'] == '30분이내'),
        (df['CKG_DODF_NM'].isin(['아코', '珂', '틜コ', '薩', '慈', '珂'])) & (df['CKG_TIME_NM'] == '60분이내'),
        (df['CKG_DODF_NM'].isin(['아코', '珂', '틜コ', '薩', '慈', '珂'])) & (df['CKG_TIME_NM'] == '90분이내')
    ]
    choices = ['초급', '초급', '중급', '고급', '고급']
    df.loc[conditions[0] | conditions[1] | conditions[2] | conditions[3] | conditions[4], 'CKG_DODF_NM'] = choices[0]
    df.loc[conditions[1], 'CKG_DODF_NM'] = choices[1]
    df.loc[conditions[2], 'CKG_DODF_NM'] = choices[2]
    df.loc[conditions[3], 'CKG_DODF_NM'] = choices[3]
    df.loc[conditions[4], 'CKG_DODF_NM'] = choices[4]

    return df

df = update_difficulty(df)

df.loc[df['CKG_DODF_NM'] == '珂', 'CKG_DODF_NM'] = '중급'
df.loc[df['CKG_DODF_NM'].isin(['珂', '아コ', '틜コ']), 'CKG_DODF_NM'] = '중급'

In [ ]:
def add_ckg_dodf_pt(df):
    """
    Adds a numerical representation of the difficulty level (CKG_DODF_PT) to the DataFrame.
    """
    df['CKG_DODF_PT'] = 0

    df.loc[df['CKG_DODF_NM'] == '아무나', 'CKG_DODF_PT'] = 1
    df.loc[df['CKG_DODF_NM'] == '초급', 'CKG_DODF_PT'] = 2
    df.loc[df['CKG_DODF_NM'] == '중급', 'CKG_DODF_PT'] = 3
    df.loc[df['CKG_DODF_NM'] == '고급', 'CKG_DODF_PT'] = 4
    df.loc[df['CKG_DODF_NM'] == '최상위', 'CKG_DODF_PT'] = 5

    return df

df = add_ckg_dodf_pt(df)

In [ ]:
df

,RCP_SNO,CKG_NM,INQ_CNT,RCMM_CNT,SRAP_CNT,CKG_MTH_ACTO_NM,CKG_STA_ACTO_NM,CKG_MTRL_ACTO_NM,CKG_KND_ACTO_NM,CKG_MTRL_CN,CKG_INBUN_NM,CKG_DODF_NM,CKG_TIME_NM,MATERIAL_CNT,SEASONING_CNT,CKG_DODF_PT
0,128671,어묵김말이,9592,6,66,튀김,간식,가공식품류,디저트,[재료] 어묵 2개| 김밥용김 3장| 당면 1움큼| 양파 1/2개| 당근 1/2개|...,2인분,초급,60분이내,10,0,2
3,131871,현미호두죽,2912,0,9,끓이기,일상,쌀,밥/죽/떡,[재료] 현미 4컵| 찹쌀 2컵| 호두 50g| 물 1/2컵| 소금 약간,2인분,초급,30분이내,5,0,2
4,139247,북어갈비,6865,3,97,굽기,술안주,건어물류,메인반찬,[재료] 북어포 1마리| 찹쌀가루 1C [양념] 간장 2T| 설탕 1T| 물 1T|...,2인분,초급,60분이내,2,8,2
5,149207,토마토스파게티,12754,2,36,볶음,일상,가공식품류,면/만두,[재료] 파스타면 [양념] 토마토 1개| 토마토 페이스트 3T| 양파 1/2개| 다...,1인분,초급,30분이내,1,6,2
6,151148,표고버섯탕수,16053,2,194,튀김,손님접대,버섯류,메인반찬,[재료] 건표고버섯 9개| 오이 1/2개| 당근 1/2개| 양파 1/2개| 사과 1...,2인분,초급,30분이내,7,8,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336578,7041366,도토리묵무침,15,0,1,무침,일상,기타,메인반찬,[재료] 도토리묵700g| 상추10장(또는 쌈채소)| 당근1/2개| 양파1/2개| ...,4인분,초급,NaN,6,6,2
336579,7041367,묵밥,8,0,0,끓이기,일상,기타,밥/죽/떡,[주재료] 도토리묵350g| 국물멸치1줌| 다시마1개| 물500ml| 멸치액젓2t|...,3인분,아무나,NaN,0,12,1
336580,7041368,세발나물무침,10,0,0,무침,일상,채소류,밑반찬,[재료] 세발나물200g| 소금0.5큰술(데칠용)| 국간장0.5큰술| 멸치액젓0.5...,4인분,초급,30분이내,7,0,2
336581,7041369,제육볶음,11,0,0,볶음,일상,돼지고기,메인반찬,[주재료] 대패삼겹살700g| 느타리버섯300g| 양파1/2개| 청양고추2개| 홍고...,4인분,아무나,30분이내,0,14,1


### TRY 1
* 중복 제거, 난이도/재료수/양념수 평균 계산
* prompt
```
dataframe 이 RCP_SNOCKG_NMINQ_CNTRCMM_CNTSRAP_CNTCKG_MTH_ACTO_NMCKG_STA_ACTO_NMCKG_MTRL_ACTO_NMCKG_KND_ACTO_NMCKG_MTRL_CNCKG_INBUN_NMCKG_DODF_NMCKG_TIME_NMMATERIAL_CNTSEASONING_CNTCKG_DODF_PT 으로 구성되어 있다

CKG_NM 값의 중복없이 1개씩으로 구성하는데,
같은 CKG_NM 에 대해 CKG_DODF_PT와 MATERIAL_CNT와 SEASONING_CNT 는 각각 평균으로 계산하고
나머지 항목은 RCP_SNO가 가장 높을 것을 선택
하는 파이썬 코드
```



In [ ]:
def process_dataframe(df):
    # 수치형 컬럼들의 평균을 계산할 그룹
    numeric_cols = ['CKG_DODF_PT', 'MATERIAL_CNT', 'SEASONING_CNT']

    # 먼저 CKG_NM별로 그룹화하여 수치형 컬럼들의 평균 계산
    avg_df = df.groupby('CKG_NM')[numeric_cols].mean().reset_index().round(2)

    # RCP_SNO가 가장 높은 행을 선택하기 위한 처리
    # CKG_NM별로 그룹화하고 RCP_SNO가 가장 높은 행의 인덱스를 가져옴
    idx = df.groupby('CKG_NM')['RCP_SNO'].idxmax()

    # 해당 인덱스의 행들을 선택 (수치형 컬럼 제외한 나머지 정보)
    other_cols = [col for col in df.columns if col not in numeric_cols]
    max_rcp_df = df.loc[idx, other_cols]

    # 두 데이터프레임을 CKG_NM을 기준으로 병합
    result = pd.merge(max_rcp_df, avg_df, on='CKG_NM')

    return result

result_try1 = process_dataframe(df)
result_try1.to_csv('recipt_postproc_try1.csv', encoding='utf-8-sig', index=False)

,RCP_SNO,CKG_NM,INQ_CNT,RCMM_CNT,SRAP_CNT,CKG_MTH_ACTO_NM,CKG_STA_ACTO_NM,CKG_MTRL_ACTO_NM,CKG_KND_ACTO_NM,CKG_MTRL_CN,CKG_INBUN_NM,CKG_DODF_NM,CKG_TIME_NM,CKG_DODF_PT,MATERIAL_CNT,SEASONING_CNT
0,7038950,명란알게맛살돌솥비빔밥,39,0,1,볶음,영양식,가공식품류,밥/죽/떡,[재료] 밥1/2공기| 김치1/2컵| 명란젓1T| 참기름1t| 게맛살1개,2인분,초급,10분이내,2.0,5.0,0.0
1,6942782,불고기숙주볶음,969,0,12,볶음,일상,채소류,밑반찬,[주 재료] 숙주 120~150g| 새송이 버섯 50~60g| 양파 1/4개| 불고...,1인분,초급,30분이내,2.0,0.0,9.0
2,6936791,소고기무국,1293,0,19,끓이기,영양식,소고기,국/탕,[재료] 소고기 사태 2근| 두부 1모| 대파 2줄기| 황태채 1/2C| 다시마| ...,6인분이상,초급,2시간이상,2.0,11.0,0.0
3,6829478,야채볼,4081,6,63,비빔,초스피드,채소류,밥/죽/떡,[재료] 연어 60g| 낫토 45g| 계란노른자| 치커리| 김| 밥 [양념] 식초 ...,1인분,아무나,10분이내,1.0,6.0,5.0
4,7003897,오이지,1024,0,7,절임,일상,채소류,밑반찬,[재료] 백오이 10개| 청양고추 3개| 굵은소금 1컵| 식초 1컵| 설탕 1/2컵...,3인분,초급,2시간이상,2.0,6.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55602,6842760,寧瀆,3588,2,28,기타,일상,쌀,밥/죽/떡,[재료] 밥 2공기| 나물 한 줌| 전 5개| 묵은지| 통깨| 검은깨| 김 가루,2인분,아무나,10분이내,1.0,7.0,0.0
55603,6920973,掠뮐,3389,0,126,끓이기,기타,쌀,밥/죽/떡,[지급 재료] 불린 쌀 100g| 소고기 20g| 불린표고버섯 1개| 대파 1조각|...,1인분,초급,30분이내,2.0,0.0,10.0
55604,6873264,羚캔ㅉ,3444,1,92,볶음,초스피드,채소류,밥/죽/떡,[재료] 양파 1개| 계란 4개| 대파 적당량 [양념] 간장 4큰술| 맛술 4큰술|...,2인분,아무나,10분이내,1.0,3.0,4.0
55605,6976372,�ス갚蝸,3033,0,18,절임,일상,채소류,김치/젓갈/장류,[열무 물김치 야채 재료] 무 500g| 당근 200g| 홍고추 5개| 청양고추 7...,6인분이상,중급,2시간이상,3.0,0.0,15.0


### TRY2
* Prompt


```
<df라는 dataframe 구성>
RCP_SNOCKG_NMINQ_CNTRCMM_CNTSRAP_CNTCKG_MTH_ACTO_NMCKG_STA_ACTO_NMCKG_MTRL_ACTO_NMCKG_KND_ACTO_NMCKG_MTRL_CNCKG_INBUN_NMCKG_DODF_NMCKG_TIME_NMMATERIAL_CNTSEASONING_CNTCKG_DODF_PT

<조건>
문자인 CKG_NM 을 중복없이 1개씩으로 재구성
같은 CKG_NM 에 대해 CKG_DODF_PT와 MATERIAL_CNT와 SEASONING_CNT 는 각각 소수점 둘째자리 평균으로 계산
나머지 항목은 RCP_SNO가 가장 높을 것을 선택

하는 파이썬 코드
```




In [ ]:
def process_dataframe(df):
    # 숫자형 컬럼들의 평균을 계산할 그룹
    avg_columns = ['CKG_DODF_PT', 'MATERIAL_CNT', 'SEASONING_CNT']

    # 평균 계산을 위한 데이터프레임
    avg_df = df.groupby('CKG_NM')[avg_columns].agg(lambda x: round(x.mean(), 2)).reset_index()

    # RCP_SNO가 가장 높은 행을 선택하기 위한 데이터프레임
    max_rcp_df = df.sort_values('RCP_SNO', ascending=False).groupby('CKG_NM').first().reset_index()

    # 평균을 계산한 컬럼을 제외한 나머지 컬럼들
    other_columns = [col for col in df.columns if col not in avg_columns]

    # 최종 결과 데이터프레임 생성
    result_df = max_rcp_df[other_columns].merge(
        avg_df,
        on='CKG_NM',
        how='left'
    )

    return result_df

# 사용 예시:
result_try2 = process_dataframe(df)
result_try2.to_csv('recipt_postproc_try2.csv', encoding='utf-8-sig', index=False)

## 테스트 코드

In [ ]:
# prompt: recipe_preproc.csv 파일을 dataframe 으로 읽어 RCP_SNO 이 비어있는 row 수를 세고 row 제거

# Load the CSV file into a pandas DataFrame.
try:
  df = pd.read_csv('recipe_preproc.csv')
except FileNotFoundError:
  print("Error: recipe_preproc.csv not found.")
  # You might want to add code here to handle the missing file,
  # such as exiting the script or prompting the user for a different file.
  exit()

# Count the number of rows where 'RCP_SNO' is empty (NaN).
empty_rcp_sno_count = df['RCP_SNO'].isnull().sum()
print(f"Number of rows with empty RCP_SNO: {empty_rcp_sno_count}")

# Remove rows where 'RCP_SNO' is empty.
df = df.dropna(subset=['RCP_SNO'])

# Print the updated DataFrame (optional).
print(df.head())